## Import

In [ ]:
import math
import matplotlib as plt
%matplotlib inline
import pandas as pd
from statsmodels.tsa.holtwinters import Holt
from sqlalchemy import create_engine

## Load Data

In [ ]:
def load_database(db_name, table, columns):
    engine = create_engine(f"sqlite:///../out/{db_name}.db", echo=False, future=False)
    db = None,
    with engine.connect() as con:
        sql_query = pd.read_sql_query(f"select * from {table}", con)
        db = pd.DataFrame(sql_query, columns = columns)
    return db
                                      
data = load_database("binance_historic_trades_1_week", "historic_trades", 
                           ["tradeId", "symbol", "price", "quantity", "timestamp"])

## Dropping Superflous Data

In [ ]:
data = data[["price", "timestamp"]]

## Data Quality Checks

In [ ]:
duplicated_rows = data[data.duplicated()]
duplicated_timestamps = data[data["timestamp"].duplicated()]


print(f"Duplicated rows: {duplicated_rows.shape[0]}")
print(f"Duplicated timestamps: {duplicated_timestamps.shape[0]}")

In [ ]:
nan_rows = data[data.isna().any(axis=1)]

print(f"NaN rows: {nan_rows.shape[0]}")

In [ ]:
diffed = data["timestamp"].diff().drop(0)
is_sorted = (diffed >= 0).all()

print(f"Dataframe timestamps are sorted = {is_sorted}")

## Data Conversions

### Convert unix time to datetime and set frequency

In [ ]:
data["timestamp"] = pd.to_datetime(data["timestamp"], unit = "ms").dt.round("min")

### Squash duplicate timestamps

In [ ]:
data = data.groupby("timestamp").agg({"price": "mean"})
data = data.reset_index()

### Rounding Price

In [ ]:
data = data.round({"price": 0})

## Timeseries Evenly Spaced

In [ ]:
diffed_stamps = data["timestamp"].dt.minute.diff().drop(0)
is_evenly_spaced = ((diffed_stamps == 1) | (diffed_stamps == -59)).all()

print(f"Timeseries is evenly spaced = {is_evenly_spaced}")

In [ ]:
data = data.set_index("timestamp")
data.resample(rule = "min")
data = data.asfreq("min")
data["price"] = data["price"].fillna(method="ffill")

In [ ]:
resetted = data.reset_index()
diffed_stamps = resetted["timestamp"].dt.minute.diff().drop(0)
is_evenly_spaced = ((diffed_stamps == 1) | (diffed_stamps == -59)).all()

print(f"Timeseries is evenly spaced = {is_evenly_spaced}")

## Differencing

In [ ]:
#data = data.diff().dropna()
#Probably not needed

## Apply Exponential Smoothing

In [ ]:
train_count = round(data.shape[0] * 0.90)
test_count = train_count + 1

train_data = data.iloc[:train_count]
test_data = data.iloc[test_count:]

In [ ]:
model = Holt(train_data["price"], exponential=True, initialization_method="estimated").fit()

In [ ]:
f = model.forecast(60*2)
f = f.to_frame(name="price")

In [ ]:
data['price'].plot(legend=True,label="Complete")
f['price'].plot(legend=True,label="Forecast",figsize=(12,8));